In [52]:
#importing req library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image

#Importing libraries for defining the architechure
import torch
import torch.nn as nn
from torch.optim import adam
from torch.nn import ReLU,BCELoss,Sequential,Sigmoid,Linear

#importing torch vision
from torchvision.models import googlenet
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader

##  Loading and pre-processing the Dataset

In [53]:
csv_path="G:\\NN\\Assment\\Emergency and non emergency vechile clasification\\Dataset\\emergency_classification.csv"
img_dir='G:/NN/Assment/Emergency and non emergency vechile clasification/Dataset/images/'

In [54]:
#defining the preprocessing steps
normalize=transforms.Normalize(mean=[0.485,0.456,0.406],
                               std=[0.229,0.224,0.225])
preprocessing=transforms.Compose([transforms.ToTensor(),normalize])

In [55]:
# defining the class to load dataset 
class EmergencyDataset(Dataset):
    """Custom Dataset for loading Emergency Dataset"""

    # defining the init function
    def __init__(self, csv_path, img_dir, transform):
        df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df.image_names.values
        self.y = df['emergency_or_not'].values
        self.transform = transform

    # defining the get item function
    def __getitem__(self, index):
        img = Image.open(self.img_dir + self.img_names[index])

        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]
        return img, label

    # defining the len function
    def __len__(self):
        return self.y.shape[0]


In [56]:
# loading the dataset
train_dataset = EmergencyDataset(csv_path,img_dir,
                              transform=preprocessing)

In [57]:
#loading the data in batch using dataloader
trainloader=DataLoader(dataset=train_dataset,
                       batch_size=45,
                       shuffle=True)


In [58]:
for batch_idx ,(batch_x,batch_y)in enumerate(trainloader):
    break

In [59]:
#shape of the label
batch_x.shape,batch_y.shape

(torch.Size([45, 3, 224, 224]), torch.Size([45]))

##  loading the weights of the pre trained model

In [60]:
# define model architecture along with pretrained weights of googlenet / inception_v1
googlenet_model=googlenet(pretrained=True)

In [61]:
#print the model
googlenet_model

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [62]:
# architecture in form of a list
list(googlenet_model.children())

[BasicConv2d(
   (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
 BasicConv2d(
   (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
 Inception(
   (branch1): BasicConv2d(
     (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   )
   (branch2): Sequential(
     (0): BasicConv2d(
       (conv): 

In [63]:
# defining the class to extract features
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.net = googlenet(pretrained=True)
        #push to cuda
        if torch.cuda.is_available():
            self.net = self.net.cuda()
        for p in self.net.parameters():
            p.requires_grad = False
        # Define which layers you are going to extract
        self.features = nn.Sequential(*list(self.net.children())[:-3])        

    def forward(self, x):
        return self.features(x)

In [64]:
# prepare input
input = batch_x[:5]
input = input.cuda()

# pass the input to vgg16
if __name__ == "__main__":
    fe = FeatureExtractor()
    output = fe(input)

# shape of the output
output.shape


torch.Size([5, 1024, 7, 7])

##  Fine tune the model for the current problem

**extract the Feature**

In [65]:
feature=[]
target=[]
time_elapsed=[]
#set model to eval
googlenet_model.eval()
#deactivate auto grad

with torch.no_grad():
    # getting the data in batches using defined data loader
    for batch_idx,(batch_x,batch_y) in enumerate (trainloader):
        if torch.cuda.is_available():
           batch_x=batch_x.cuda()
        #record time for extracting the features
        start=torch.cuda.Event(enable_timing=True)
        end=torch.cuda.Event(enable_timing=True)

        start.record()
        #extracting the features
        if __name__== "__main__":
            fe=FeatureExtractor()
            batch_features=fe(batch_x)

        end.record()
        #waits for everything to finish
        torch.cuda.synchronize()
        #time elapsed
        time_elapsed.append(start.elapsed_time(end))
        #converting to numpy
        batch_features=batch_features.data.cpu().numpy()
        # append in list
        feature.append(batch_features)
        target.append(batch_y)

#save to the array
feature = np.concatenate(feature, axis=0)
target = np.concatenate(target, axis=0)
    


In [66]:
# time taken to extract features
print('Time taken in seconds: ', torch.sum(torch.tensor(time_elapsed))*0.001)

Time taken in seconds:  tensor(10.0763)


In [67]:
# shape of the features
feature.shape

(2352, 1024, 7, 7)

**Flatting the data**

In [68]:
#flattening the features
feature=feature.reshape(len(feature),-1)
feature.shape

(2352, 50176)

In [69]:
# creating the training and validation data
X_train, X_valid, y_train, y_valid = train_test_split(feature, target, test_size=0.3, stratify=target, random_state=42)
# shape of training and validation set
(X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape)

(((1646, 50176), (1646,)), ((706, 50176), (706,)))

In [70]:
# converting training and validation set to PyTorch tensor
X_train=torch.FloatTensor(X_train)
y_train=torch.FloatTensor(y_train)
X_valid=torch.FloatTensor(X_valid)
y_valid=torch.FloatTensor(y_valid)

**Defining the NN**

In [71]:
model=Sequential(Linear(1024*7*7,64),
                  ReLU(),
                  Linear(64,1),
                  Sigmoid()
                  )

In [72]:
#summarry
model

Sequential(
  (0): Linear(in_features=50176, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=1, bias=True)
  (3): Sigmoid()
)

In [73]:
# pass an input to the model to understand the output
model(X_train[0].view(1,1024*7*7))

tensor([[0.4488]], grad_fn=<SigmoidBackward>)

**Compile model**

In [74]:
# define optimizer and loss function
optimizer=torch.optim.Adam(model.parameters())
criterion= nn.BCELoss()
#checking for gpu
if torch.cuda.is_available():
    model=model.cuda()
    criterion=criterion.cuda()


In [75]:
#define metric
def binary_accuracy(preds, y):
  
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)

    #no. of correctly classified    
    correct = (rounded_preds == y).float()

    #compute accuracy 
    acc = correct.sum() / len(correct)
    return acc

In [76]:
# define training function
def train(X,y,batch_size):

  #activate training phase
  model.train()
  
  #initialization
  epoch_loss, epoch_acc= 0, 0
  no_of_batches = 0

  #randomly create indices
  indices= torch.randperm(len(X))
  
  #loading in batches
  for i in range(0,len(indices),batch_size):
    
    #indices for a batch
    ind = indices[i:i+batch_size]
  
    #batch  
    batch_x=X[ind]
    batch_y=y[ind]
    
    #push to cuda
    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    #clear gradients
    optimizer.zero_grad()
          
    #forward pass
    outputs = model(batch_x)

    #converting to a 1 dimensional tensor
    outputs = outputs.squeeze()

    #calculate loss and accuracy
    loss = criterion(outputs, batch_y)
    acc = binary_accuracy(outputs, batch_y)  
    
    #Backward pass
    loss.backward()
    
    #Update weights
    optimizer.step()

    #Keep track of the loss and accuracy of a epoch
    epoch_loss = epoch_loss + loss.item()
    epoch_acc  = epoch_acc  + acc.item()

    #No. of batches
    no_of_batches = no_of_batches+1

  return epoch_loss/no_of_batches, epoch_acc/no_of_batches

In [77]:
# define evaluation function
def evaluate(X,y,batch_size):

  #deactivate training phase
  model.eval()

  #initialization
  epoch_loss, epoch_acc= 0, 0
  no_of_batches = 0

  #randomly create indices
  indices= torch.randperm(len(X))

  #deactivates autograd
  with torch.no_grad():
    
    #loading in batches
    for i in range(0,len(indices),batch_size):
      
      #indices for a batch
      ind = indices[i:i+batch_size]
  
      #batch  
      batch_x= X[ind]
      batch_y= y[ind]

      #push to cuda
      if torch.cuda.is_available():
          batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
      #Forward pass
      outputs = model(batch_x)

      #converting the output to 1 Dimensional tensor
      outputs = outputs.squeeze()

      # Calculate loss and accuracy
      loss = criterion(outputs, batch_y)
      acc = binary_accuracy(outputs, batch_y)   
      
      #keep track of loss and accuracy of an epoch
      epoch_loss = epoch_loss + loss.item()
      epoch_acc  = epoch_acc  + acc.item()

      #no. of batches
      no_of_batches = no_of_batches + 1

    return epoch_loss/no_of_batches, epoch_acc/no_of_batches

In [78]:
# define prediction function
def predict(X,batch_size):
  
  #deactivate training phase
  model.eval()

  # initialization 
  predictions = []

  # create indices
  indices = torch.arange(len(X))

  #deactivates autograd
  with torch.no_grad():
      
      for i in range(0, len(X), batch_size):
        
        #indices for a batch
        ind = indices[i:i+batch_size]

        # batch
        batch_x = X[ind]

        #push to cuda
        if torch.cuda.is_available():
            batch_x = batch_x.cuda()

        #Forward pass
        outputs = model(batch_x)

        #converting the output to 1 Dimensional tensor
        outputs = outputs.squeeze()

        # convert to numpy array
        prediction = outputs.data.cpu().numpy()
        predictions.append(prediction)
    
  # convert to single numpy array
  predictions = np.concatenate(predictions, axis=0)
    
  return predictions

In [79]:
N_EPOCHS = 10
batch_size = 32

# intialization
best_valid_acc = 0

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc  = train(X_train, y_train, batch_size)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(X_valid, y_valid, batch_size)

    print('\nEpoch :',epoch,
          'Training loss:',round(train_loss,4),
          '\tTrain Accuracy:',round(train_acc,4),
          '\tValidation loss:',round(valid_loss,4),
          '\tValidation Accuracy:',round(valid_acc,4))

    #save the best model
    if best_valid_acc <= valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'saved_weights.pt') 
        print("\n----------------------------------------------------Saved best model------------------------------------------------------------------")  


Epoch : 0 Training loss: 0.5341 	Train Accuracy: 0.845 	Validation loss: 0.2164 	Validation Accuracy: 0.9117

----------------------------------------------------Saved best model------------------------------------------------------------------

Epoch : 1 Training loss: 0.0912 	Train Accuracy: 0.9698 	Validation loss: 0.1768 	Validation Accuracy: 0.9348

----------------------------------------------------Saved best model------------------------------------------------------------------

Epoch : 2 Training loss: 0.0506 	Train Accuracy: 0.9862 	Validation loss: 0.2137 	Validation Accuracy: 0.9307

Epoch : 3 Training loss: 0.0163 	Train Accuracy: 0.9964 	Validation loss: 0.2776 	Validation Accuracy: 0.9117

Epoch : 4 Training loss: 0.014 	Train Accuracy: 0.9976 	Validation loss: 0.2656 	Validation Accuracy: 0.9198

Epoch : 5 Training loss: 0.0076 	Train Accuracy: 0.9994 	Validation loss: 0.2152 	Validation Accuracy: 0.9307

Epoch : 6 Training loss: 0.0098 	Train Accuracy: 0.9994 	Valida

In [80]:
#load weights of best model
path='saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [81]:
valid_loss, valid_accuracy = evaluate(X_valid,y_valid,batch_size)

print("Validation Accuracy:",(valid_accuracy)*100)

Validation Accuracy: 93.88586956521739
